# Notebook to Calculate Turbulence Properties for OpenFoam

In this notebook I will collect all the calculations have to be done to set up the **Initial and Boundary Conditions** for _OpenFOAM_.

These are mostly set up only for **Internal (Pipe) flow**.

### 1. Pipe Dimensions and Given Properties of Flow

**The properties are:**\
Pipe Diameter, _D(m)_ \
Reynolds Number, _Re_ \
Dynamic Viscosity, $\mu$ (Pa.s) \
Density, $\rho$ (kg/m^3) \

---
Viscosity and desity for a nominal pressure and Temperature can be found from different sources. For Steam, [Spirax Sacro](https://www.spiraxsarco.com/resources-and-design-tools/steam-tables/superheated-steam-region) is a high fidelity source to get steam properties.

In [1]:
D = 0.1 # hydraulic diameter
Re = 1e7 # Re
mu = 3.33393e-5 # from Spirax Sacro
rho = 36.1221 # from Spirx Sacro

Therefore the _free-stream velocity_ or _Reynolds Averaged Mean Velocity_, $U$ is calculated from: $Re = \frac{\rho UD}{\mu}$ 
$$\therefore U=\frac{\mu Re}{\rho D}$$

In [2]:
U = (mu*Re)/(rho*D)
print("The Free-Stream Velociy, U = ", U)
print("Kinematic Viscosity nu = ", mu/rho)

The Free-Stream Velociy, U =  92.29612896260184
Kinematic Viscosity nu =  9.229612896260184e-07


### 2. Estimate of approximated turbulence parameters
**Which are used to calculate $k, \omega, \epsilon$ etc.** \
Turbulence Intensity(fully developed, mean over area), $$I = 0.227Re^{-0.1}$$
Turbulence Intensity(fully developed, axis) $$I = 0.055Re^{-0.0407}$$        
Ref: [CFD Online Wiki for Turbulence Intesity](https://www.cfd-online.com/Wiki/Turbulence_intensity)

Length Scale (mixing length), $$Tu_L(m) = 0.038D$$                                  
Ref: [CFD-Online Wiki for Length Scale](https://www.cfd-online.com/Wiki/Turbulence_length_scale)

In [3]:
I = 0.227*Re**(-0.1)
l = 0.038*D
q = rho*U*3.14159265359*(D/2)**2

print("The Turbulence Intensity I = ", I*100, "%.\n")
print("The Length Scale, l = ", l, "m.\n")
print("Flow Rate, Q = ", q, " kg/s")

The Turbulence Intensity I =  4.529245454979357 %.

The Length Scale, l =  0.0038 m.

Flow Rate, Q =  26.18462498895828  kg/s


**The Turbulence Intesity Can Also be Calculated Using The Following Script (Roughness can be added)**\
Ref: https://www.mdpi.com/2311-5521/4/4/180

In [4]:
"""

N.T.Basse 2019

Based on paper:
    Turbulence Intensity Scaling: A Fugue
    https://www.mdpi.com/2311-5521/4/4/180

"""

import numpy as np
from scipy.optimize import fsolve

#smooth friction factor (Eq. 19 in paper)
def smooth(x):
    out = [np.power(x[0],-0.5)-1.930*np.log10(Re*np.sqrt(x[0]))+0.537]
    return out

#rough friction factor (Eq. 20 in paper)
def rough(x):
    out = [np.power(x[0],-0.5)+2*np.log10(k_s_norm/(2*3.7)+2.51/(Re*np.sqrt(x[0])))]          
    return out

#user supplies Reynolds number Re [dimensionless]
Re=Re

#user supplies pipe radius a [m]
a=D/2

#user supplies sand-grain roughness k_s [m]
k_s=0.0

#normalized sand-grain roughness is calculated [dimensionless]
k_s_norm=k_s/a

#calculate friction factor
if k_s==0:
    print('Smooth pipe')
    ff = fsolve(smooth, 0.01)
else:
    print('Rough pipe')
    ff = fsolve(rough,0.01)
    
print('friction factor:')    
print(ff)   

#calculate turbulence intensity (Eq. 29 in paper)
TI=0.0276*np.log(ff)+0.1794
    
print('turbulence intensity:')
print(TI)

Smooth pipe
friction factor:
[0.0083162]
turbulence intensity:
[0.04720843]


### 3. Now Use These Parameters To Get The Other Turbulence Properties

[This website from CFD-online tools](https://www.cfd-online.com/Tools/turbulence.php) is very helpful to calculate these properties. Just select the [Turbulence variables (k, ε, ω) from turbulence intensity (Tu), length-scale (TuL) and freestream velocity (U∞)](https://www.cfd-online.com/Tools/turbulence.php#:~:text=Turbulence%20variables%20(k%2C%20%CE%B5%2C%20%CF%89)%20from%20turbulence%20intensity%20(Tu)%2C%20length%2Dscale%20(TuL)%20and%20freestream%20velocity%20(U%E2%88%9E)) and put the previously caltulated values there and press calculate.

**The website can be used from below.**

In [5]:
print("Needed Parameters: \n\t Free-Stream Velocity: ", U,"\n\t","Turbulence Intensity: ", I*100,"\n\t","Length Scale:\t\t", l, "\n\t", "Kinematic Viscosity: ", mu/rho)

Needed Parameters: 
	 Free-Stream Velocity:  92.29612896260184 
	 Turbulence Intensity:  4.529245454979357 
	 Length Scale:		 0.0038 
	 Kinematic Viscosity:  9.229612896260184e-07


In [6]:
%%html
<iframe src="https://www.cfd-online.com/Tools/turbulence.php" width="1000" height="600"></iframe>

**Insert the values of $k, \epsilon, \omega$ below from the above website.**

In [7]:
# Insert the nessessary parameters from above
k = 26.21259070795487
epsilon = 3178.511831293987
omega = 1347.3219232302354

### 4. Calculate $\mu_t$ or `nut` of OF

Firstly, get $\mu_t/\mu$ from the above website using $k, \omega$, or $\epsilon$ got from previous step.\
_Note that, here $\mu$ is [molecular dynamic viscosity](https://www.cfd-online.com/Wiki/Eddy_viscosity_ratio) or [Kinematic Viscosity](https://www.cfd-online.com/Forums/fluent/229919-difference-between-molecular-dynamic-viscosity.html) which is contradictory to common notation._ And `nut` in `0` directory is [νt	=	Turbulent viscosity at y+ [m2/s]](https://www.openfoam.com/documentation/guides/latest/doc/guide-bcs-wall-turbulence-nutWallFunction.html#:~:text=where-,%CE%BD,%5Bm2/s%5D,-%CE%BD) with similar unit as $\nu$.

[This cfd-online thread](https://www.cfd-online.com/Forums/openfoam/230490-initialization-nut.html) focuses on estimation of `nut`. It can be infered that `nut` as calculated at boundaries is preferable and value used in internalField is not important for steady-state analysis but crucial for transient analysis.

#### Ways of estimating `nut` for internalField

Calculate $$\nu_t = (\nu_t/\nu)\nu$$ \
or $$\nu_t = \frac{(\nu_t/\nu)\mu}{\rho}$$

**OR using the following formula.**
$$\nu_t = \rho C \mu \frac{k}{\epsilon}$$

In [8]:
# here the 1st formula is used

# insert (nut/nu) or eddy viscosity ratio
edvisRatio = 21079.24654058659 # from above website
nut = edvisRatio*mu/rho
print("nut for internalField using the 1st formula is ", nut)

nut for internalField using the 1st formula is  0.019455328571444587


## Optional: For Supersonic Flow
Total Pressure / Stagnant Pressure Calculation for Supersonic Flow. The equations are from [here](https://www.grc.nasa.gov/www/BGH/isentrop.html).

In [9]:
# gamma = cp/cv
cp = 2565.52
cv = 1874.87
c = 693.266 # from spirax sacro
gamma = cp/cv

Temp = 873
RR = 8.314 #[J/mol.K]
RR = 8.314/(18e-3) #[J/kg.K]
p = 1.35e7

M = U/np.sqrt(gamma*RR*Temp)
p0 = p*(1 + ((gamma-1)/2)*M*M)**(gamma/(gamma-1))

T0 = Temp*(1+ (gamma -1)*M*M/2)

print("Total Pressure: ", p0, "\nTotal Temperature: ", T0)
print("\nMach Number = \t", M, "\nSound Velocity (from Spirax Sacro) = \t", c, "\nSound Velocity (from above equation) = ", U/M )

Total Pressure:  13643151.107419318 
Total Temperature:  875.4824589383526

Mach Number = 	 0.12425259987216147 
Sound Velocity (from Spirax Sacro) = 	 693.266 
Sound Velocity (from above equation) =  742.8104446712715


# End